In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

2025-06-28 07:53:51.729186: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('data/train.csv')
data.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [5]:
soil_encoder = LabelEncoder()
crop_encoder = LabelEncoder()
ferti_encoder = LabelEncoder()


In [6]:
data['Soil Type'] = soil_encoder.fit_transform(data['Soil Type'])
data['Crop Type'] = crop_encoder.fit_transform(data['Crop Type'])
data['Fertilizer Name'] = ferti_encoder.fit_transform(data['Fertilizer Name'])
data.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,1,8,36,4,5,4
1,1,27,69,65,4,4,30,6,18,4
2,2,29,63,32,4,4,24,12,16,2
3,3,35,62,54,4,0,39,12,4,0
4,4,35,58,43,3,6,37,2,16,5


In [7]:
X = data.drop(['Fertilizer Name', 'id'], axis = 1)
y = data['Fertilizer Name']

In [8]:
X 

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,37,70,36,1,8,36,4,5
1,27,69,65,4,4,30,6,18
2,29,63,32,4,4,24,12,16
3,35,62,54,4,0,39,12,4
4,35,58,43,3,6,37,2,16
...,...,...,...,...,...,...,...,...
749995,25,69,30,1,3,8,16,6
749996,37,64,58,2,8,38,8,20
749997,35,68,59,4,2,6,11,29
749998,31,68,29,3,1,9,11,12


In [9]:
# scaler = StandardScaler()
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
X_train

array([[0.23076923, 0.04545455, 0.55      , ..., 0.42105263, 0.89473684,
        0.57142857],
       [0.61538462, 0.54545455, 0.125     , ..., 0.07894737, 0.        ,
        0.14285714],
       [1.        , 0.40909091, 0.4       , ..., 0.52631579, 0.57894737,
        1.        ],
       ...,
       [0.38461538, 0.        , 0.65      , ..., 0.        , 0.36842105,
        0.35714286],
       [1.        , 0.13636364, 0.575     , ..., 0.73684211, 0.10526316,
        0.83333333],
       [0.38461538, 0.27272727, 0.7       , ..., 0.18421053, 0.21052632,
        0.26190476]])

In [12]:
input_shape = [X_train.shape[1]]
input_shape

[8]

In [13]:
num_classes = len(np.unique(y))
num_classes

7

In [14]:
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

### Model Building 

In [15]:
model = keras.Sequential([
    layers.BatchNormalization(input_shape = input_shape),
    layers.Dense(64, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation = 'relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(num_classes, activation = 'relu'),
])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/normalization/batch_normalization.py:142: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [17]:
ealy_stopping = keras.callbacks.EarlyStopping(
    patience = 10,
    min_delta = 0.001,
    restore_best_weights = True,
)

In [ ]:
model.fit(
    X_train, 
    y_train_cat, 
    validation_data = (X_test, y_test_cat), 
    epochs = 50, 
    batch_size = 32, 
    callbacks = [ealy_stopping],
    verbose = 1,
)

Epoch 1/50
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 109s 6ms/step - accuracy: 0.1512 - loss: nan - val_accuracy: 0.1523 - val_loss: nan
Epoch 2/50
18745/18750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1520 - loss: nan

In [28]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

  24/4688 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step   

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/ops/nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


4688/4688 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/ops/nn.py:827: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


In [29]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.15
